In [1]:
# Import dependencies
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

In [2]:
# Generate a new set of 1,500 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lats_lngs = zip(lats, lngs)
lats_lngs

In [3]:
# Add the latitudes and longitudes in a list.
coordinates = list(lats_lngs)
print (coordinates)

[(-37.385896996780104, 2.1328245494396185), (45.3761565839464, -161.6929268168499), (-75.25610772874964, 72.94538199694105), (-40.08814513721035, -177.10927987826943), (-60.150600627488494, -93.11445034362605), (-62.49877177873385, 168.06779109465384), (-42.33266008549617, 91.44575179546831), (-86.38844693989097, 21.53678407349898), (24.275501724969004, 179.81106133771237), (-87.35459876713311, 167.61474186446054), (88.04955062257642, -3.9221284756511636), (-14.804730176917914, -149.30681473809668), (-37.76337348806818, -171.39929609602467), (63.50163587509533, 119.58217954020671), (-4.887888922353866, -98.06201720046411), (71.51153195858998, 162.75491610336655), (-55.59127660900436, -67.49631745541664), (71.65041285796968, 152.34776360222378), (38.390621868682246, 94.85304800530884), (-51.14373150721296, -10.039865690526455), (-14.498675026330744, -163.82210042237955), (33.04606015297213, -45.570994727904974), (78.30886807249638, 119.82650003245527), (52.98629058594773, 66.11149020032

In [4]:
# import the citipy module.
from citipy import citipy

In [5]:
#Create a list for holding the names of cities.
cities = []

#Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
#If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

597

In [6]:
# Import the requests library.
import requests

#Import the API key.
from config import weather_api_key

In [7]:
# Starting URL for weather map API call.
url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [8]:
#Import the datetime module from the datetime library.
from datetime import datetime

In [9]:
#Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 ==0 and i>=50):
        set_count += 1
        record_count = 1
        
    #Create endpoint url for each city.
    city_url = url + "&q=" + city
        
    #log the url,record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
        
    #Add 1 to the record count.
    record_count += 1
        
    #Run an API request for each of the cities.
    try:
         #Parse the JSON and retrieve the data.
        city_weather = requests.get(city_url).json()
        #Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description=city_weather["weather"][0]["description"]
            
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        #Rain data
        try:
            city_rain = city_weather["rain"]["3h"]
            
        except:
            city_rain = 0.00 
            
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0.00
            
            
        #Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                            "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,                                                         
                             "Current Description":city_weather_description,
                              "Rain inches (last 3hrs)":city_rain,
                                "Snow inches (last 3hrs)":city_snow})
    #If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
#Indicate the data loading is complete.
print("------------------------------")
print("Data Retrieval Complete       ")
print("------------------------------")       

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | saldanha
Processing Record 2 of Set 1 | bethel
Processing Record 3 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 4 of Set 1 | vaini
Processing Record 5 of Set 1 | punta arenas
Processing Record 6 of Set 1 | bluff
Processing Record 7 of Set 1 | busselton
Processing Record 8 of Set 1 | bredasdorp
Processing Record 9 of Set 1 | butaritari
Processing Record 10 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 11 of Set 1 | fare
Processing Record 12 of Set 1 | verkhnevilyuysk
Processing Record 13 of Set 1 | puerto ayora
Processing Record 14 of Set 1 | cherskiy
Processing Record 15 of Set 1 | ushuaia
Processing Record 16 of Set 1 | srednekolymsk
Processing Record 17 of Set 1 | yumen
Processing Record 18 of Set 1 | cape town
Processing Record 19 of Set 1 | satitoa
City not found. Skipping...
Processing Record 20 of Set 1 | torbay
Processing Record 21 of Set 1 | sa

In [13]:
len(city_data)

548

In [14]:
# Create a city dataframe.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,Saldanha,ZA,2022-07-29 02:52:29,-33.0117,17.9442,57.25,81,57,18.79,broken clouds,0.0,0.0
1,Bethel,US,2022-07-29 02:51:02,41.3712,-73.4140,79.48,94,88,4.52,overcast clouds,0.0,0.0
2,Vaini,TO,2022-07-29 02:50:52,-21.2000,-175.2000,80.76,89,20,8.05,few clouds,0.0,0.0
3,Punta Arenas,CL,2022-07-29 02:50:40,-53.1500,-70.9167,39.31,87,40,26.46,scattered clouds,0.0,0.0
4,Bluff,NZ,2022-07-29 02:50:50,-46.6000,168.3333,47.80,86,99,20.38,overcast clouds,0.0,0.0
5,Busselton,AU,2022-07-29 02:50:39,-33.6500,115.3333,60.66,61,90,21.99,overcast clouds,0.0,0.0
6,Bredasdorp,ZA,2022-07-29 02:46:05,-34.5322,20.0403,55.74,62,27,11.68,scattered clouds,0.0,0.0
7,Butaritari,KI,2022-07-29 02:51:03,3.0707,172.7902,82.08,73,58,14.52,broken clouds,0.0,0.0
8,Fare,PF,2022-07-29 02:53:30,-16.7000,-151.0167,79.09,74,0,21.07,clear sky,0.0,0.0
9,Verkhnevilyuysk,RU,2022-07-29 02:54:48,63.4453,120.3167,86.50,38,0,10.87,clear sky,0.0,0.0


In [15]:
#Export the dataframe to csv file.
city_data_df.to_csv("Weather_Database/WeatherPy_challenge.csv", index_label="City ID")

In [16]:
# Number of Cities recorded with snowfall and rainfall.
cities_rain_snow_count = len(city_data_df[(city_data_df["Rain inches (last 3hrs)"] > 0) | (city_data_df["Rain inches (last 3hrs)"] > 0)])

cities_rain_snow_count

0